# Preprocessing and Drawing the Virtual Board
In this file, we are going to:
1. Mask and detect the edge of the board
2. Deskew the rotated board
3. Draw pegs on the deskew board
4. Create the virtual board with electrical components on it

In [140]:
import numpy as np
from PIL import Image, ImageDraw,ImageFont
import cv2
from ultralytics import YOLO

In [5]:
def whatAngle(boardBW):
    
	coords = np.column_stack(np.where(boardBW > 0))
	angle = cv2.minAreaRect(coords)[-1]

	if angle < -45:
		angle = -(90 + angle)
	 
	else:
		angle = -angle
		
	return angle


def tilt(image, angle):
	# rotate the image to deskew it
	(h, w) = image.shape[:2]
	center = (w // 2, h // 2)
	M = cv2.getRotationMatrix2D(center, angle, 1.0)
	rotated = cv2.warpAffine(image, M, (w, h),
	flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

	return rotated


def get_edges(image):

	minRow = float('inf')
	minColumn = float('inf')
	maxRow = float('-inf')
	maxColumn = float('-inf')
	
	for row in range(0,len(image)):
		for column in range(0,len(image[0])):
			pixelValue = image[row][column]
			if pixelValue > 0:
				if minRow > row:
					minRow = row
				if maxRow < row:
					maxRow = row
				if minColumn > column:
					minColumn = column
				if maxColumn < column:
					maxColumn = column
					
	extLeft = minColumn
	extRight = maxColumn
	extTop = minRow
	extBot = maxRow
    
	return (extLeft, extRight, extBot, extTop)

In [6]:
def get_boundary(img):
        x,y,w,h = 0,0,0,0
        contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area>500:
                cv2.drawContours(img, cnt, -1, (255, 0, 0), 3)
                peri = cv2.arcLength(cnt,True)
                approx = cv2.approxPolyDP(cnt,0.0009*peri,True)
                x, y, w, h = cv2.boundingRect(approx)
                # cv2.imshow('contour_result',img)
                # cv2.waitKey(1)

        return x,y,w,h

In [7]:
def get_board_mask(img):
    color = [0, 0, 0, 255, 255, 50]
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array(color[:3])  
    upper = np.array(color[3:]) 
    mask = cv2.inRange(imgHSV, lower, upper)
    return mask

In [38]:
def get_pegs(img,x1,x2,y1,y2):
    matrixcoor_to_realcoor = {}
    dist_from_edge = [(x2-x1)/13,(y2-y1)/15]
    board_width = x2-x1-2*dist_from_edge[0]
    board_height = y2-y1-2*dist_from_edge[1]
    horizontal_interval = board_width / 12
    vertical_interval = board_height / 14
    img_circle = img.copy()
    
    # relate matrix coordinate to real peg coordinate
    for i in range(0,13):
        for j in range(0,15):
            matrixcoor_to_realcoor[i,j] = np.array([x1 + int(horizontal_interval * i + dist_from_edge[0]), y1 + int(vertical_interval * j + dist_from_edge[1])])
    
    for key in matrixcoor_to_realcoor:
        cv2.circle(img_circle, matrixcoor_to_realcoor[key], 2, (200, 200, 200), -1)
    
    return img_circle,matrixcoor_to_realcoor


In [68]:
def draws_pegs_on_rotated_board(image,draw_edge=False):
    boardBW = get_board_mask(image)
    angle = whatAngle(boardBW)
    boardBW_tilt = tilt(boardBW, angle) 
    image_tilt = tilt(image, angle) 
    cv2.imwrite('board_deskew.png',image_tilt)  
            
    #get the max edges of the board then draw edges and pegs on it
    x1,x2,y1,y2 = get_edges(boardBW_tilt)
    img_circle, matrixcoor_to_realcoor = get_pegs(image_tilt,x1,x2,y1,y2)
    
    if draw_edge:
        cv2.rectangle(img_circle, (x1, y1), (x2, y2), (0, 255, 0), 3)
    cv2.imwrite('board_with_pegs.png',img_circle)    
    return matrixcoor_to_realcoor

In [87]:
def round_to_integer_with_error(float_number, error_rate = 0.1, down = True):
    if down:
        lower_integer = int(float_number)

        # Calculate the error between the float number and the lower integer
        error = float_number - lower_integer

        # Check if the error is within the custom error rate
        if error <= error_rate:
            return lower_integer - 1
        else:
            return lower_integer 
    else:
        upper_integer = np.ceil(float_number).astype(int)

        # Calculate the error between the float number and the upper integer
        error = upper_integer - float_number

        # Check if the error is within the custom error rate
        if error <= error_rate:
            return upper_integer + 1
        else:
            return upper_integer
        

In [114]:
def matrix_class_mapping(results,matrixcoor_to_realcoor):
    x0,y0 = matrixcoor_to_realcoor[(0,14)]
    matrix = np.zeros((15, 13))-1

    for r in results:
        x1,y1,x2,y2,class_id = r
        grid_x1 = round_to_integer_with_error(((x1-x0) / x_len) * 12,down=False)
        grid_y1 = round_to_integer_with_error(((y1-y0) / y_len) * 14,down=False)
        grid_x2 = round_to_integer_with_error(((x2-x0) / x_len) * 12)
        grid_y2 = round_to_integer_with_error(((y2-y0) / y_len) * 14)

        grid_x1 = max(0, min(grid_x1, 12 - 1))
        grid_y1 = max(0, min(grid_y1, 14 - 1))
        grid_x2 = max(0, min(grid_x2, 12 - 1))
        grid_y2 = max(0, min(grid_y2, 14 - 1))

        matrix[grid_x1:grid_x2+1,grid_y1:grid_y2+1]=class_id
    return matrix

In [125]:
color_mapping = {
    0: 'red',
    1: 'green',
    2: 'blue',
    3: 'yellow',
    4: 'orange',
    5: 'tal',
    6: 'cyan',
    7: 'magenta',
    8: 'brown',
    9: 'pink',
    10: 'gray',
    11: 'lime',
    12: 'purple',
    13: 'olive'
}

In [138]:
def show_estimated_board(results_transferred,color_mapping=color_mapping,rows = 15,cols = 13,cell_size = 50):
    """Draw the virtual image of the board

    Args:
        results_transferred (matrix): a matrix to store class of each block of the board
        rows (int, optional): number of rows of the grid. Defaults to 8.
        cols (int, optional): number of columns of the grid. Defaults to 7.
        cell_size (int, optional): size of cell. Defaults to 50.
    """

    # Calculate the total size of the image
    image_width = cols * cell_size
    image_height = rows * cell_size

    # Create a new image with a black background
    image = Image.new("RGB", (image_width, image_height), color="black")

    # Create a draw object
    draw = ImageDraw.Draw(image)


    # Draw the grid with numbers
    for row in range(rows):
        for col in range(cols):
            # Calculate the position of the top-left corner of the cell
            x1 = col * cell_size
            y1 = row * cell_size

            # Calculate the position of the bottom-right corner of the cell
            x2 = x1 + cell_size
            y2 = y1 + cell_size

            # Calculate the number for each cell (you can use any logic here)
            cell_number = results_transferred[row][col]

            # Draw the cell with the corresponding number
            if cell_number >= 0:
                draw.rectangle([x1, y1, x2, y2], fill=color_mapping[cell_number],outline='white')
            else:
                draw.rectangle([x1, y1, x2, y2], fill="black",outline='white')
            draw.text((x1 + 20, y1 + 20), str(cell_number),  fill="white")

    # Save the image to a file
    image.save("grid_image_with_numbers.png")

    # Alternatively, display the image using the default image viewer
    image.show()

Implement on data:

In [139]:
# Replace 'image_path.jpg' with the actual path to your image file
image_path = 'raw10.png'

# Read the image using cv2.imread()
image = cv2.imread(image_path)

# implementation
matrixcoor_to_realcoor = draws_pegs_on_rotated_board(image)

model = YOLO('best (5).pt')
image = cv2.imread('board_deskew.png')
results = model.predict(image)

for result in results:
        boxes = result.boxes
        #print(np.shape(boxes.xyxy),np.shape(boxes.cls[:,np.newaxis]))
        output = np.hstack([boxes.xyxy,boxes.cls[:,np.newaxis]])
        print(output)
        #print([names[i] for i in boxes.cls.tolist()])

# implement show_estimated_board
matrix = matrix_class_mapping(output,matrixcoor_to_realcoor)

show_estimated_board(matrix)


0: 672x800 1 battery, 1 led, 1 switch, 1 wire, 191.6ms
Speed: 2.4ms preprocess, 191.6ms inference, 0.8ms postprocess per image at shape (1, 3, 672, 800)


[[     410.51       400.1      512.54      433.76           6]
 [     378.52      300.33      421.71      433.56          12]
 [     386.56      228.59      515.62      337.34           0]
 [     485.28      305.27       514.6      427.49          13]]
